## Packages and paths

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
from datetime import datetime

In [2]:
data_source_path = 'data/source/'
data_output_path = 'data/'

## Static definitions

In [3]:
target_countries = ['Iceland', 'Ireland', 'France', 'Portugal', 'Switzerland']

donor_countries = ['Austria', 'Belgium', 'Bulgaria', 
#                    'Cyprus', 
                   'Croatia', 'Czech Republic',
                   'Germany', 'Greece', 'Hungary', 'Italy', 'Lithuania', 'Netherlands', 
                   'Romania', 'Slovakia', 'Spain']

incl_countries = target_countries + donor_countries
incl_countries.sort()

print(len(incl_countries))
print(incl_countries)

19
['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Czech Republic', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Lithuania', 'Netherlands', 'Portugal', 'Romania', 'Slovakia', 'Spain', 'Switzerland']


In [4]:
incl_years = range(2000, 2020)

print(list(incl_years))

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


## Functions

In [5]:
def month_to_num(month):
    return {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9, 
            'Oct': 10,
            'Nov': 11,
            'Dec': 12
    }[month]

In [6]:
def quarter_to_month(quarter):
    if quarter == 1:
        month = 1
    elif quarter == 2:
        month = 4
    elif quarter == 3:
        month = 7
    elif quarter == 4:
        month = 12
    return month

## Monthly CO2 data

In [7]:
# read
file_name = 'co2_m'
co2_raw = pd.read_csv(f'{data_source_path}{file_name}.csv')
co2_raw = co2_raw[co2_raw.columns.drop(list(co2_raw.filter(regex='Unnamed')))]
co2_raw

# select
co2 = co2_raw.copy()
co2 = co2[(co2['Name'].isin(incl_countries))
          & (co2['Year'].isin(incl_years))]

# pivot
co2 = co2.melt(id_vars=['Name', 'Year'],value_vars=co2.columns[2:])
co2['Month'] = co2.apply(lambda row: month_to_num(row.variable), axis=1)
co2['date'] = pd.to_datetime(dict(year=co2.Year, month=co2.Month, day=1))
co2 = co2.drop('variable', axis=1)

co2 = co2[['Name', 'date', 'value']]
co2.columns = ['country', 'date', 'co2']
co2 = co2.sort_values(by=['country', 'date'])
co2['co2'] = co2['co2']*1000
co2 = co2.reset_index(drop=True)

print(len(co2['country'].unique()))
print(co2['country'].unique())

co2.to_csv(f'{data_output_path}{file_name}.csv')

co2

19
['Austria' 'Belgium' 'Bulgaria' 'Croatia' 'Czech Republic' 'France'
 'Germany' 'Greece' 'Hungary' 'Iceland' 'Ireland' 'Italy' 'Lithuania'
 'Netherlands' 'Portugal' 'Romania' 'Slovakia' 'Spain' 'Switzerland']


,country,date,co2
0,Austria,2000-01-01,9726927.646
1,Austria,2000-02-01,8532685.059
2,Austria,2000-03-01,8134567.013
3,Austria,2000-04-01,6546737.758
4,Austria,2000-05-01,5603261.736
...,...,...,...
4555,Switzerland,2019-08-01,2793634.688
4556,Switzerland,2019-09-01,3445179.636
4557,Switzerland,2019-10-01,3819263.405
4558,Switzerland,2019-11-01,4487169.820


## Quarterly Population Data

In [8]:
file_name = 'population_quarterly'
measure = ['PERSA']
power = 1000

# read
pop_raw = pd.read_csv(f'{data_source_path}{file_name}.csv')
pop_raw

# transform
pop = pop_raw.copy()
pop = pop.replace({'Slovak Republic': 'Slovakia'})
pop['year'] = pop['TIME'].str[:4].astype(int)
pop['quarter'] = pop['TIME'].str[6:].astype(int)
pop['month'] = pop.apply(lambda row: quarter_to_month(row.quarter), axis=1)
pop['date'] = pd.to_datetime(dict(year=pop.year, month=pop.month, day=1)).dt.to_period('M')
pop['Value_cor'] = pop['Value']*power
pop

# select
pop = pop[(pop['Country'].isin(incl_countries))
          & (pop['year'].isin(incl_years))
          & (pop['MEASURE'].isin(measure))]
pop

# pivot
pop = pop[['Country', 'date', 'Value_cor']]
pop.columns = ['country', 'date', 'population']
pop = pop.sort_values(by=['country', 'date'])
pop = pop.reset_index(drop=True)
pop

print(len(pop['country'].unique()))
print(pop['country'].unique())

pop.to_csv(f'{data_output_path}{file_name}.csv')

pop

19
['Austria' 'Belgium' 'Bulgaria' 'Croatia' 'Czech Republic' 'France'
 'Germany' 'Greece' 'Hungary' 'Iceland' 'Ireland' 'Italy' 'Lithuania'
 'Netherlands' 'Portugal' 'Romania' 'Slovakia' 'Spain' 'Switzerland']


,country,date,population
0,Austria,2000-01,8003000.0
1,Austria,2000-04,8008100.0
2,Austria,2000-07,8014100.0
3,Austria,2000-12,8021000.0
4,Austria,2001-01,8028700.0
...,...,...,...
1511,Switzerland,2018-12,8542300.0
1512,Switzerland,2019-01,8555500.0
1513,Switzerland,2019-04,8570100.0
1514,Switzerland,2019-07,8586500.0


In [9]:
file_name = 'population_monthly'

pop_resample = pd.DataFrame({'population' : [],
                             'country' : []})

for country in pop['country'].unique():
    pop_country = pop[pop['country']==country]
    pop_country = pop_country.set_index('date')['population']
    pop_country = pop_country.resample('M').interpolate().to_frame()
    pop_country['country'] = [country]*len(pop_country)
    pop_resample = pd.concat([pop_resample, pop_country], axis=0)

pop_resample = pop_resample.reset_index().rename(columns={'index': 'date'})
pop_resample['date'] = pd.to_datetime(pop_resample['date'].astype(str))
pop_resample = pop_resample[['country', 'date', 'population']]

pop.to_csv(f'{data_output_path}{file_name}.csv')

pop_resample

,country,date,population
0,Austria,2000-01-01,8003000.0
1,Austria,2000-02-01,8004700.0
2,Austria,2000-03-01,8006400.0
3,Austria,2000-04-01,8008100.0
4,Austria,2000-05-01,8010100.0
...,...,...,...
4555,Switzerland,2019-08-01,8589980.0
4556,Switzerland,2019-09-01,8593460.0
4557,Switzerland,2019-10-01,8596940.0
4558,Switzerland,2019-11-01,8600420.0


## Quarterly GDP Data

In [11]:
file_name = 'gdp_q'
measure = ['CPCARSA']
power = 1e6

# read
gdp_raw = pd.read_csv(f'{data_source_path}{file_name}.csv')
gdp_raw

# transform
gdp = gdp_raw.copy()
gdp = gdp.replace({'Slovak Republic': 'Slovakia'})
gdp['year'] = gdp['TIME'].str[:4].astype(int)
gdp['quarter'] = gdp['TIME'].str[6:].astype(int)
gdp['month'] = gdp.apply(lambda row: quarter_to_month(row.quarter), axis=1)
gdp['date'] = pd.to_datetime(dict(year=gdp.year, month=gdp.month, day=1)).dt.to_period('M')
gdp['Value_cor'] = gdp['Value']*power
gdp

# select
gdp = gdp[(gdp['Country'].isin(incl_countries))
          & (gdp['year'].isin(incl_years))
          & (gdp['MEASURE'].isin(measure))]
gdp

# pivot
gdp = gdp[['Country', 'date', 'Value_cor']]
gdp.columns = ['country', 'date', 'gdp']
gdp = gdp.sort_values(by=['country', 'date', 'gdp'])
gdp = gdp.reset_index(drop=True)
gdp

print(len(gdp['country'].unique()))
print(gdp['country'].unique())

gdp.to_csv(f'{data_output_path}{file_name}.csv')

gdp

19
['Austria' 'Belgium' 'Bulgaria' 'Croatia' 'Czech Republic' 'France'
 'Germany' 'Greece' 'Hungary' 'Iceland' 'Ireland' 'Italy' 'Lithuania'
 'Netherlands' 'Portugal' 'Romania' 'Slovakia' 'Spain' 'Switzerland']


,country,date,gdp
0,Austria,2000-01,2.309478e+11
1,Austria,2000-04,2.344696e+11
2,Austria,2000-07,2.367135e+11
3,Austria,2000-12,2.395027e+11
4,Austria,2001-01,2.377669e+11
...,...,...,...
1515,Switzerland,2018-12,6.086313e+11
1516,Switzerland,2019-01,6.137651e+11
1517,Switzerland,2019-04,6.214928e+11
1518,Switzerland,2019-07,6.270779e+11


In [12]:
file_name = 'population_monthly'

pop_resample = pd.DataFrame({'population' : [],
                             'country' : []})

for country in pop['country'].unique():
    pop_country = pop[pop['country']==country]
    pop_country = pop_country.set_index('date')['population']
    pop_country = pop_country.resample('M').interpolate().to_frame()
    pop_country['country'] = [country]*len(pop_country)
    pop_resample = pd.concat([pop_resample, pop_country], axis=0)

pop_resample = pop_resample.reset_index().rename(columns={'index': 'date'})
pop_resample['date'] = pd.to_datetime(pop_resample['date'].astype(str))
pop_resample = pop_resample[['country', 'date', 'population']]

pop.to_csv(f'{data_output_path}{file_name}.csv')

pop_resample

,country,date,population
0,Austria,2000-01-01,8003000.0
1,Austria,2000-02-01,8004700.0
2,Austria,2000-03-01,8006400.0
3,Austria,2000-04-01,8008100.0
4,Austria,2000-05-01,8010100.0
...,...,...,...
4555,Switzerland,2019-08-01,8589980.0
4556,Switzerland,2019-09-01,8593460.0
4557,Switzerland,2019-10-01,8596940.0
4558,Switzerland,2019-11-01,8600420.0


In [13]:
file_name = 'gdp_m'

gdp_resample = pd.DataFrame({'gdp' : [],
                             'country' : []})

for country in gdp['country'].unique():
    gdp_country = gdp[gdp['country']==country]
    gdp_country = gdp_country.set_index('date')['gdp']
    gdp_country = gdp_country.resample('M').interpolate().to_frame()
    gdp_country['country'] = [country]*len(gdp_country)
    gdp_resample = pd.concat([gdp_resample, gdp_country], axis=0)
    
gdp_resample = gdp_resample.reset_index().rename(columns={'index': 'date'})
gdp_resample['date'] = pd.to_datetime(gdp_resample['date'].astype(str))
gdp_resample = gdp_resample[['country', 'date', 'gdp']]

gdp_resample.to_csv(f'{data_output_path}{file_name}.csv')

gdp_resample

,country,date,gdp
0,Austria,2000-01-01,2.309478e+11
1,Austria,2000-02-01,2.321217e+11
2,Austria,2000-03-01,2.332957e+11
3,Austria,2000-04-01,2.344696e+11
4,Austria,2000-05-01,2.352176e+11
...,...,...,...
4555,Switzerland,2019-08-01,6.281104e+11
4556,Switzerland,2019-09-01,6.291429e+11
4557,Switzerland,2019-10-01,6.301753e+11
4558,Switzerland,2019-11-01,6.312078e+11


## Total join

In [14]:
file_name = 'total'

total = co2.copy()
total = total.merge(pop_resample, how='left', on=['date', 'country'])
total = total.merge(gdp_resample, how='left', on=['date', 'country'])

total['co2_cap'] = total['co2'] / total['population']
total['gdp_cap'] = total['gdp'] / total['population']

total.to_csv(f'{data_output_path}{file_name}.csv', header=True, index=False)

total

,country,date,co2,population,gdp,co2_cap,gdp_cap
0,Austria,2000-01-01,9726927.646,8003000.0,2.309478e+11,1.215410,28857.653380
1,Austria,2000-02-01,8532685.059,8004700.0,2.321217e+11,1.065959,28998.180236
2,Austria,2000-03-01,8134567.013,8006400.0,2.332957e+11,1.016008,29138.647415
3,Austria,2000-04-01,6546737.758,8008100.0,2.344696e+11,0.817514,29279.054957
4,Austria,2000-05-01,5603261.736,8010100.0,2.352176e+11,0.699525,29365.122366
...,...,...,...,...,...,...,...
4555,Switzerland,2019-08-01,2793634.688,8589980.0,6.281104e+11,0.325220,73121.285498
4556,Switzerland,2019-09-01,3445179.636,8593460.0,6.291429e+11,0.400907,73211.821548
4557,Switzerland,2019-10-01,3819263.405,8596940.0,6.301753e+11,0.444258,73302.284301
4558,Switzerland,2019-11-01,4487169.820,8600420.0,6.312078e+11,0.521738,73392.673846


## Stationarity

In [15]:
var = 'co2'
test_adf = total.copy()
test_adf = np.log(test_adf[test_adf['country']=='Netherlands'][var]).diff(1)

print(adfuller(test_adf.dropna()))
plt.plot(test_adf)

(-6.4183162354864995, 1.8178212517158938e-08, 12, 226, {'1%': -3.4596204846395824, '5%': -2.8744153028455948, '10%': -2.5736320761218576}, -752.5497160232583)


## Total stationary

In [16]:
file_name = 'total_stat'

total_stat = total.copy()

total_stat = total_stat[['country', 'date', 'co2', 'population', 'gdp', 'co2_cap', 'gdp_cap']]

# co2_diff = 12
# pop_diff = 13
# gdp_diff = 13
# co2_cap_diff = 13
# gdp_cap_diff = 13
# max_diff = max(co2_diff, pop_diff, gdp_diff, co2_cap_diff, gdp_cap_diff)

country_list = []
date_list = []
co2_stat_list = []
pop_stat_list = []
gdp_stat_list = []
co2_cap_stat_list = []
gdp_cap_stat_list = []


for country in total_stat['country'].unique():
    total_sub = total_stat[total_stat['country']==country]
    
    country_list += list(total_sub['country'])
    date_list += list(total_sub['date'])
    
    for series in ['co2', 'population', 'gdp', 'co2_cap', 'gdp_cap']:
        total_sub_sub = total_sub[series]
        
        if series == 'co2':
            co2_stat_list += list(np.log(total_sub_sub).diff(12))
        elif series == 'population':
            pop_stat_list += list(np.log(total_sub_sub).diff(1).diff(1))
        elif series == 'gdp':
            gdp_stat_list += list(np.log(total_sub_sub).diff(1).diff(1))
        elif series == 'co2_cap':
            co2_cap_stat_list += list(np.log(total_sub_sub).diff(1).diff(1))
        elif series == 'gdp_cap':
            gdp_cap_stat_list += list(np.log(total_sub_sub).diff(1).diff(1))
            
total_stat = pd.DataFrame(list(zip(country_list, date_list, co2_stat_list, pop_stat_list, gdp_stat_list, co2_cap_stat_list, gdp_cap_stat_list)), 
                          columns =['country', 'date', 'co2_stat', 'pop_stat', 'gdp_stat', 'co2_cap_stat', 'gdp_cap_stat'])

total_stat = total_stat.dropna(axis=0, how='any').reset_index(drop=True)

total_stat.to_csv(f'{data_output_path}{file_name}.csv', header=True, index=False)

total_stat

,country,date,co2_stat,pop_stat,gdp_stat,co2_cap_stat,gdp_cap_stat
0,Austria,2001-01-01,-0.019138,7.874564e-04,-9.605785e-03,-0.012303,-1.039324e-02
1,Austria,2001-02-01,0.039611,-6.066812e-04,7.105802e-03,-0.173589,7.712484e-03
2,Austria,2001-03-01,0.022160,-1.244510e-07,-2.826436e-08,0.007013,9.618666e-08
3,Austria,2001-04-01,0.119718,-1.243633e-07,-2.827386e-08,-0.054365,9.608939e-08
4,Austria,2001-05-01,0.024326,3.718831e-05,1.385755e-03,-0.131449,1.348566e-03
...,...,...,...,...,...,...,...
4327,Switzerland,2019-08-01,0.027670,-2.316558e-04,-1.328125e-03,-0.003615,-1.096470e-03
4328,Switzerland,2019-09-01,-0.000445,-1.641248e-07,-2.702041e-06,0.226123,-2.537916e-06
4329,Switzerland,2019-10-01,0.008483,-1.639919e-07,-2.693180e-06,-0.106551,-2.529188e-06
4330,Switzerland,2019-11-01,-0.004228,-1.638592e-07,-2.684362e-06,0.058083,-2.520503e-06
